# Make RGB NIRCam stamps showing slit footprint
We will be using the `trilogy` package to make RGB stamps and the shutter footprint regions.
The information we need to show the footprint and the source (optional) is all in the "shutters" files.


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np

from astropy.io import fits
from miri_utils.stamp_maker import resample_nircam, make_stamp

In [ ]:
# ---------- SETUP ----------
input_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/"
output_dir = "/home/bpc/University/master/Red_Cardinal/stamps_miri_v2/"
os.makedirs(output_dir, exist_ok=True)

Now let's resample the NIRCam data instead of MIRI!

NB: The transformation of the WCS does not work properly, but we are not concerned about it here since we only use the images to stack them together anyways, we don't care about sky coordinates here.

Call the resampling function:

In [ ]:
resample_nircam(input_dir, 27)

In [ ]:
# ---------- STEP 1: Get IDs from NIRCam, PRIMER and COSMOS-Web ----------
f444w_files = glob.glob(os.path.join(input_dir, "*F444W_cutout_nircam_res.fits"))
f444w_ids = [os.path.basename(f).split("_F444W")[0] for f in f444w_files]

f770w_primer_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_primer.fits"))
f770w_primer_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_primer_files]

f770w_cweb_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_cweb.fits"))
f770w_cweb_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_cweb_files]

f1800w_files = glob.glob(os.path.join(input_dir, "*F1800W*.fits"))
f1800w_ids = [os.path.basename(f).split("_F1800W")[0] for f in f1800w_files]

# Remove all galaxies in f770w_primer_ids from f770w_cweb_ids
f770w_cweb_ids = np.setdiff1d(f770w_cweb_ids, f770w_primer_ids)
f770w_leftover_primer = np.setdiff1d(f770w_primer_ids, f1800w_ids)

Now let's try and call the function:

In [ ]:
for gal_id in f1800w_ids:
    try:
        print(f"Creating image for {gal_id}...")
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F1800W_cutout_primer.fits")
        g_file_primer_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_primer.fits")
        g_file_cweb_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb.fits")
        b_file_base = os.path.join(input_dir, f"{gal_id}_F444W_cutout_nircam_res.fits")

        # Decide which F770W file to use
        if gal_id in f770w_primer_ids and os.path.exists(g_file_primer_base):
            g_file = g_file_primer_base + "[1]"
        elif gal_id in f770w_cweb_ids and os.path.exists(g_file_cweb_base):
            g_file = g_file_cweb_base + "[1]"
        else:
            print(f"[Skipping {gal_id}] No valid F770W file found.")
            continue

        # Now safely add [0] to files
        r_file = r_file_base + "[1]"
        b_file = b_file_base + "[0]"
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': [g_file], 
                    'B': [b_file]}
        
        # Call your image function (without shutters)
        outfile = os.path.join(output_dir, f"{gal_id}_primer.pdf")
        if gal_id == '19098':
            make_stamp(imagesRGB, outfile=outfile, 
                    Q_r=10, alpha_r=1, weight_r=1,
                    Q_g=10, alpha_g=1, weight_g=1,
                    Q_b=10, alpha_b=5, weight_b=1,
                    stretch='asinh')
            
        else:
            make_stamp(imagesRGB, outfile=outfile, 
                    Q_r=5, alpha_r=0.7, weight_r=1,
                    Q_g=5, alpha_g=0.85, weight_g=1,
                    Q_b=10, alpha_b=1.0, weight_b=1,
                    stretch='asinh')

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")

Produce the stamps for all PRIMER galaxies that were only mapped in F770W AND for COSMOS-Web

In [ ]:
for gal_id in f770w_leftover_primer:
    try:
        print(f"Creating image for {gal_id}...")
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_primer.fits")
        b_file_base = os.path.join(input_dir, f"{gal_id}_F444W_cutout_nircam_res.fits")

        # Now safely add [1] to files
        r_file = r_file_base + "[1]"
        b_file = b_file_base + "[0]"
        
        with fits.open(r_file_base) as hdul:
            r_data = hdul[1].data
            g_data = np.zeros_like(r_data)  # fake green
        fits.writeto('fake_green.fits', g_data, overwrite=True)
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': ['fake_green.fits[0]'], 
                    'B': [b_file]}
        
        # Call your image function (without shutters)
        # Q determines the strength of the non-linearity for the stretch.
        # alpha adjusts the sensitivity of the stretch for the given filter.

        outfile = os.path.join(output_dir, f"{gal_id}_primer.pdf")
        make_stamp(imagesRGB, outfile=outfile, 
                Q_r=5, alpha_r=0.7, weight_r=1,
                Q_g=5, alpha_g=0.85, weight_g=1,
                Q_b=10, alpha_b=1.0, weight_b=1,
                stretch='asinh')

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")

for gal_id in f770w_cweb_ids:
    try:
        print(f"Creating image for {gal_id}...")
        
        # Construct base filenames (without [0])
        r_file_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb.fits")
        b_file_base = os.path.join(input_dir, f"{gal_id}_F444W_cutout_nircam_res.fits")

        # Now safely add [1] to files
        r_file = r_file_base + "[1]"
        b_file = b_file_base + "[0]"
        
        with fits.open(r_file_base) as hdul:
            r_data = hdul[1].data
            g_data = np.zeros_like(r_data)  # fake green
        fits.writeto('fake_green.fits', g_data, overwrite=True)
        
        # Stack into imagesRGB dictionary
        imagesRGB = {'R': [r_file], 
                    'G': ['fake_green.fits[0]'], 
                    'B': [b_file]}
        
        # Call your image function (without shutters)
        # Q determines the strength of the non-linearity for the stretch.
        # alpha adjusts the sensitivity of the stretch for the given filter.

        outfile = os.path.join(output_dir, f"{gal_id}_cweb.pdf")
        make_stamp(imagesRGB, outfile=outfile, 
                Q_r=5, alpha_r=0.7, weight_r=1,
                Q_g=5, alpha_g=0.85, weight_g=1,
                Q_b=10, alpha_b=1.0, weight_b=1,
                stretch='asinh')

    except Exception as e:
        print(f"Error processing {gal_id}: {e}")

Check extensions in cutout FITS files

In [ ]:
r_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F1800W_cutout_primer.fits"
with fits.open(r_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")

g_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F770W_cutout_primer.fits"
with fits.open(g_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")
        
b_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F444W_cutout_nircam.fits"
with fits.open(b_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")


Now let's check if we got all the galaxies:

In [ ]:
files = glob.glob(os.path.join(input_dir, "*.fits"))
unique_ids = []

for file in files:
    filename = os.path.basename(file)
    
    if "nircam" in filename:     
        continue
    
    id = filename.split("_")[0]
    if id not in unique_ids:
        unique_ids.append(id)
print(unique_ids)
print(len(unique_ids))

stamps = glob.glob(os.path.join(output_dir, "*.pdf"))
unique_stamps = []

for stamp in stamps:
    stampname = os.path.basename(stamp)
    id = stampname.split("_")[0]
    if id not in unique_stamps:
        unique_stamps.append(id)

print(unique_stamps)
print(len(unique_stamps))

for gal in f770w_leftover_primer:
    if gal in f770w_cweb_ids:
        print(gal)
